## Lab3 Goal 1.1 CNN Solution

Modifies the model used of Goal 1.1 FNN solution to a CNN (Convolutional Neural Network)

This model typically achieves superior accuracy, but requires more resources for training.

In [51]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [52]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [53]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [54]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model (Convoution Neural Network (CNN) model)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),  # Input: 1×28×28 → Output: 32×28×28
            nn.ReLU(),
            nn.MaxPool2d(2, 2),                         # Output: 32×14×14

            nn.Conv2d(32, 64, kernel_size=3, padding=1),# Output: 64×14×14
            nn.ReLU(),
            nn.MaxPool2d(2, 2)                          # Output: 64×7×7
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),                               # Flatten 64×7×7 → 3136
            nn.Linear(64 * 7 * 7, 512),
            nn.ReLU(),
            nn.Linear(512, 10)                          # 10 output classes
        )

    def forward(self, x):
        x = self.conv_stack(x)
        x = self.classifier(x)
        return x

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (conv_stack): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [55]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [56]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [57]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [58]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298105  [   64/60000]
loss: 2.296595  [ 6464/60000]
loss: 2.289516  [12864/60000]
loss: 2.288608  [19264/60000]
loss: 2.280614  [25664/60000]
loss: 2.264225  [32064/60000]
loss: 2.264091  [38464/60000]
loss: 2.246985  [44864/60000]
loss: 2.240802  [51264/60000]
loss: 2.229663  [57664/60000]
Test Error: 
 Accuracy: 52.2%, Avg loss: 2.225539 

Epoch 2
-------------------------------
loss: 2.226297  [   64/60000]
loss: 2.220311  [ 6464/60000]
loss: 2.186768  [12864/60000]
loss: 2.186873  [19264/60000]
loss: 2.142511  [25664/60000]
loss: 2.101580  [32064/60000]
loss: 2.078992  [38464/60000]
loss: 2.008898  [44864/60000]
loss: 1.962668  [51264/60000]
loss: 1.861062  [57664/60000]
Test Error: 
 Accuracy: 54.9%, Avg loss: 1.838639 

Epoch 3
-------------------------------
loss: 1.871634  [   64/60000]
loss: 1.786209  [ 6464/60000]
loss: 1.602075  [12864/60000]
loss: 1.571836  [19264/60000]
loss: 1.365254  [25664/60000]
loss: 1.339514  [32064/600

In [59]:
torch.save(model.state_dict(), "model_CNN.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [60]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model_CNN.pth", weights_only=True))

<All keys matched successfully>

In [72]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    #x = x.to(device)
    x = x.unsqueeze(0).to(device)  # Shape becomes [1, 1, 28, 28]
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

9
Predicted: "Ankle boot", Actual: "Ankle boot"


In [ ]:
training_data